In [ ]:
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst/sst.mnmean.v4.nc'
ds = xr.open_dataset(url)
ds

In [ ]:
ds = ds.sel(time=slice('1960', '2018'))
ds

In [ ]:
# size in MB
ds.nbytes/1e6 

In [ ]:
# actually download the data
ds.load()

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [ ]:
dataset = hv.Dataset(ds.sst)
hv_im = (dataset.to(hv.Image, ['lon', 'lat'], dynamic=True)
                .redim.range(sst=(-2, 30))
                .options(cmap='Magma', width=800, height=450, colorbar=True))

%output holomap='scrubber' fps=3
regrid(hv_im, precompute=True)

In [ ]:
sst_clim = ds.sst.groupby('time.month').mean(dim='time')
sst_anom = ds.sst.groupby('time.month') - sst_clim

In [ ]:
dataset = hv.Dataset(sst_anom)
hv_im = (dataset.to(hv.Image, ['lon', 'lat'], dynamic=True)
                .redim.range(sst=(-2, 2))
                .options(cmap='RdBu_r', width=800, height=450, colorbar=True))

%output holomap='scrubber' fps=3
regrid(hv_im, precompute=True)

In [ ]:
sst_anom_nino34 = sst_anom.sel(lat=slice(5, -5), lon=slice(190, 240))
sst_anom_nino34_mean = sst_anom_nino34.mean(dim=('lon', 'lat'))
oni = sst_anom_nino34_mean.rolling(time=3, center=True).mean()

![NOAA Nino 3.4](https://camo.githubusercontent.com/f4fb57e583e664d9b5f1861d4b062ad0a32aeac1/68747470733a2f2f7777772e6e6364632e6e6f61612e676f762f6d6f6e69746f72696e672d636f6e74656e742f74656c65636f6e6e656374696f6e732f656c6e2d662d70672e676966)

In [ ]:
plt.grid()
oni.plot()
plt.ylabel('Anomaly (dec. C)');

In [ ]:
positive_oni = ((oni>0.5).astype('b').rolling(time=5, center=True).sum()==5)
negative_oni = ((oni<0.5).astype('b').rolling(time=5, center=True).sum()==5)
positive_oni.astype('i4').plot(marker='.')
(-negative_oni.astype('i4')).plot(marker='.')

In [ ]:
sst_anom.where(positive_oni).mean(dim='time').plot()

In [ ]:
sst_anom.where(negative_oni).mean(dim='time').plot()